## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [1]:
from pandas import (
    DataFrame, HDFStore
)
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [2]:
project_path =  "/content/drive/My Drive/DogBreed_Classification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [3]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [4]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [5]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [6]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [7]:
labels = pd.read_csv('./labels.csv')

In [8]:
labels.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


### Print the count of each category of Dogs given in the dataset



In [9]:
print(labels.breed.value_counts())

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
komondor                 67
golden_retriever         67
brabancon_griffon        67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64


### Get one-hot encodings of labels

In [10]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [12]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [13]:
labels_ec = tf.keras.utils.to_categorical(labels['breed'].factorize()[0])

In [14]:
labels_ec.shape

(10222, 120)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [15]:
from tqdm import tqdm
import cv2
img_rows = 128
img_cols = 128
x_train = []
y_train = []

for f,img in tqdm(labels.values):
  train_img = cv2.imread('./train/{}.jpg'.format(f),1)
  train_img_resize = cv2.resize(train_img,(img_rows, img_cols))
  x_train.append(train_img_resize)
  y_train.append(img)

100%|██████████| 10222/10222 [00:26<00:00, 379.76it/s]


In [16]:
print(y_train[110])
print(x_train[110].shape)

saluki
(128, 128, 3)


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [17]:
dum = pd.get_dummies(y_train)
encoded_labels = dum
y_train = dum

import numpy as np
y_train = np.array(y_train)

x_train = np.array(x_train)
x_train[0].shape

(128, 128, 3)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [18]:
from sklearn.model_selection import train_test_split
x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
print (len(x_train2))
print (len(x_val))

8177
2045


In [19]:
x_train2[0].shape

(128, 128, 3)

In [20]:
x_train2 = x_train2.reshape(x_train2.shape[0],128,128,3)
x_val = x_val.reshape(x_val.shape[0],128,128,3)
x_train2 = x_train2/255.

x_val = x_val/255.

print (x_train2.shape)
print (x_val.shape)


print(y_train2.shape)
print(y_val.shape)

(8177, 128, 128, 3)
(2045, 128, 128, 3)
(8177, 120)
(2045, 120)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [21]:
test_img = pd.read_csv('./sample_submission.csv')
test_img.sample(10)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
3004,49de8ff5d5e6ab332356610bcb042948,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
9088,e1fa4eb8a57184773ac11efe0de11e92,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4468,6cf486696a155c9e41397e18aa58f144,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1317,1f503f2c112adff1bd9ccc23c1377c59,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008

Run the below code to load the test image files in x_test_feature

In [23]:
x_test_feature = []
img_rows = 128
img_cols = 128
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f[0]), 1)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:26<00:00, 387.37it/s]


Normalize the test data and convert it into 4 dimensions

In [24]:
x_test_feature = np.array(x_test_feature)
x_test_feature[0].shape

(128, 128, 3)

In [25]:
x_test_feature = x_test_feature.reshape(x_test_feature.shape[0],128,128,3)
x_test_feature = x_val/255.
print (x_test_feature.shape)

(2045, 128, 128, 3)


### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [26]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential()

In [27]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5),input_shape=(128, 128, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(120, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Use batch_size = 128 and epochs = 10 and execute the model

In [28]:
print (x_train2.shape)
print (x_val.shape)
print(y_train2.shape)
print(y_val.shape)

(8177, 128, 128, 3)
(2045, 128, 128, 3)
(8177, 120)
(2045, 120)


In [29]:
model.fit(x_train2,
          y_train2,          
          validation_data=(x_val,y_val),
          epochs=10,
          batch_size=128)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 19s 2ms/sample - loss: 6.5151 - acc: 0.0111 - val_loss: 4.7846 - val_acc: 0.0103
Epoch 2/10
8177/8177 [==============================] - 9s 1ms/sample - loss: 4.7305 - acc: 0.0201 - val_loss: 4.7400 - val_acc: 0.0166
Epoch 3/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 4.0708 - acc: 0.1375 - val_loss: 4.8528 - val_acc: 0.0328
Epoch 4/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 1.7370 - acc: 0.6131 - val_loss: 6.5872 - val_acc: 0.0264
Epoch 5/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.2581 - acc: 0.9502 - val_loss: 9.2650 - val_acc: 0.0293
Epoch 6/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0631 - acc: 0.9941 - val_loss: 9.5597 - val_acc: 0.0274
Epoch 7/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0226 - acc: 0.9980 - val_loss: 10.2662 - val_acc: 

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [30]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False, 
    samplewise_center=False,  
    featurewise_std_normalization=False,  
    samplewise_std_normalization=False,  
    zca_whitening=False,  
    rotation_range=50,  
    width_shift_range=0.2, 
    height_shift_range=0.2,  
    horizontal_flip=True,  
    vertical_flip=True)  
datagen.fit(x_train2)

Using TensorFlow backend.


In [31]:
model.fit(x_train2,
          y_train2,          
          validation_data=(x_val,y_val),
          epochs=10,
          batch_size=128)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0152 - acc: 0.9989 - val_loss: 8.8782 - val_acc: 0.0298
Epoch 2/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0109 - acc: 0.9990 - val_loss: 10.2616 - val_acc: 0.0259
Epoch 3/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0119 - acc: 0.9991 - val_loss: 9.3449 - val_acc: 0.0264
Epoch 4/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0080 - acc: 0.9990 - val_loss: 10.3343 - val_acc: 0.0284
Epoch 5/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0078 - acc: 0.9991 - val_loss: 9.1303 - val_acc: 0.0269
Epoch 6/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0061 - acc: 0.9991 - val_loss: 9.4497 - val_acc: 0.0274
Epoch 7/10
8177/8177 [==============================] - 10s 1ms/sample - loss: 0.0113 - acc: 0.9988 - val_loss: 9.2374 - val_acc

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [32]:
train_generator = datagen.flow(x_train2,y_train2,batch_size=32)
val_generator = datagen.flow(x_val,y_val,batch_size=32)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [33]:
model.fit_generator(train_generator,steps_per_epoch=x_train2.shape[0],
                    epochs=10,
                    validation_data=val_generator)

Epoch 1/10
8177/8177 [==============================] - 986s 121ms/step - loss: 4.5476 - acc: 0.0306 - val_loss: 4.4362 - val_acc: 0.0494
Epoch 2/10
8177/8177 [==============================] - 980s 120ms/step - loss: 4.1453 - acc: 0.0731 - val_loss: 4.4448 - val_acc: 0.0538
Epoch 3/10
8177/8177 [==============================] - 977s 119ms/step - loss: 3.9265 - acc: 0.1007 - val_loss: 4.4449 - val_acc: 0.0660
Epoch 4/10
8177/8177 [==============================] - 980s 120ms/step - loss: 3.7690 - acc: 0.1222 - val_loss: 4.4168 - val_acc: 0.0748
Epoch 5/10
8177/8177 [==============================] - 984s 120ms/step - loss: 3.6524 - acc: 0.1399 - val_loss: 4.4323 - val_acc: 0.0841
Epoch 6/10
8177/8177 [==============================] - 982s 120ms/step - loss: 3.5659 - acc: 0.1532 - val_loss: 4.7046 - val_acc: 0.0822
Epoch 7/10
8177/8177 [==============================] - 979s 120ms/step - loss: 3.4974 - acc: 0.1646 - val_loss: 4.7281 - val_acc: 0.0758
Epoch 8/10
8177/8177 [============

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [35]:
tl_path = "/content/drive/My Drive/Colab Notebooks/"

In [36]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(tl_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [37]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [41]:
from keras.layers import Flatten, Dense
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(120, activation="softmax")(x)

### Make all the layers in the base_model (VGG16) to be non-trainable

In [42]:
for layer in base_model.layers[:]:
    layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [43]:
from keras import applications
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
model_final = Model(input = base_model.input, output = predictions)
sgd = optimizers.SGD(lr=0.000001, decay=1e-6, momentum=0.9, nesterov=True)
model_final.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])
checkpoint = ModelCheckpoint("vgg16_best.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
epochs=10
model_final.fit(x_train, y_train, batch_size = 128, epochs = epochs, validation_data=(x_val, y_val), callbacks = [checkpoint, early])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':



Train on 10222 samples, validate on 2045 samples
Epoch 1/10
10222/10222 [==============================] - 27s 3ms/step - loss: 19.3410 - accuracy: 0.0094 - val_loss: 4.8378 - val_accuracy: 0.0054
Epoch 2/10
  128/10222 [..............................] - ETA: 13s - loss: 17.8651 - accuracy: 0.0156

/tensorflow-1.15.2/python3.6/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/tensorflow-1.15.2/python3.6/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


10222/10222 [==============================] - 17s 2ms/step - loss: 17.7589 - accuracy: 0.0087 - val_loss: 4.8334 - val_accuracy: 0.0064
Epoch 3/10
10222/10222 [==============================] - 17s 2ms/step - loss: 16.6960 - accuracy: 0.0093 - val_loss: 4.8298 - val_accuracy: 0.0068
Epoch 4/10
10222/10222 [==============================] - 18s 2ms/step - loss: 15.8795 - accuracy: 0.0089 - val_loss: 4.8267 - val_accuracy: 0.0059
Epoch 5/10
10222/10222 [==============================] - 18s 2ms/step - loss: 15.2051 - accuracy: 0.0083 - val_loss: 4.8240 - val_accuracy: 0.0064
Epoch 6/10
10222/10222 [==============================] - 18s 2ms/step - loss: 14.6195 - accuracy: 0.0089 - val_loss: 4.8216 - val_accuracy: 0.0059
Epoch 7/10
10222/10222 [==============================] - 18s 2ms/step - loss: 14.0957 - accuracy: 0.0089 - val_loss: 4.8195 - val_accuracy: 0.0073
Epoch 8/10
10222/10222 [==============================] - 18s 2ms/step - loss: 13.6197 - accuracy: 0.0089 - val_loss: 4.817